# Qualicum Beach GCP Analysis

This notebook demonstrates how to:
1. Parse ground control points from a KMZ file
2. Download a basemap from OpenStreetMap
3. Visualize GCPs overlaid on the basemap


## Setup and Imports


In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add package to path
package_dir = Path.cwd()
sys.path.insert(0, str(package_dir))

from qualicum_beach_gcp_analysis import (
    load_gcps_from_kmz,
    download_basemap,
    visualize_gcps_on_basemap,
)
from qualicum_beach_gcp_analysis.visualization import calculate_gcp_bbox

print("Imports successful!")


## 1. Load Ground Control Points from KMZ File


In [ ]:
# Path to the KMZ file
kmz_path = "/Users/mauriciohessflores/Documents/Code/Data/Qualicum Beach GCPs/Spexi_Survey_Points/Spexi_Drone_Survey/QualicumBeach_AOI.kmz"

# Debug: Inspect KMZ structure if needed
# Uncomment the next line to see the structure of the KMZ file
# from qualicum_beach_gcp_analysis import inspect_kmz_structure
# inspect_kmz_structure(kmz_path)

# Load GCPs
gcps = load_gcps_from_kmz(kmz_path)

print(f"\nLoaded {len(gcps)} ground control points")

# Display first few GCPs
if gcps:
    print("\nFirst few GCPs:")
    for i, gcp in enumerate(gcps[:5]):
        print(f"  {i+1}. {gcp.get('id', 'Unknown')}: ({gcp['lat']:.6f}, {gcp['lon']:.6f})")
else:
    print("\n⚠️  No GCPs found! Try uncommenting the inspect_kmz_structure line above to debug.")


## 2. Calculate Bounding Box from GCPs


In [ ]:
# Calculate bounding box with padding
bbox = calculate_gcp_bbox(gcps, padding=0.01)
min_lat, min_lon, max_lat, max_lon = bbox

print(f"Bounding box:")
print(f"  Latitude: {min_lat:.6f} to {max_lat:.6f}")
print(f"  Longitude: {min_lon:.6f} to {max_lon:.6f}")
print(f"  Span: {max_lat - min_lat:.6f}° lat, {max_lon - min_lon:.6f}° lon")


## 3. Download Basemap


In [ ]:
# Create output directory
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

# Download basemap (OpenStreetMap)
basemap_path = str(output_dir / "qualicum_beach_basemap.tif")

print("Downloading basemap from OpenStreetMap...")
basemap_path = download_basemap(
    bbox=bbox,
    output_path=basemap_path,
    source="openstreetmap",
    zoom=None  # Auto-calculate zoom level
)

print(f"\nBasemap saved to: {basemap_path}")


## 4. Visualize GCPs on Basemap


In [ ]:
# Create visualization
visualization_path = str(output_dir / "qualicum_beach_gcps_visualization.png")

visualize_gcps_on_basemap(
    gcps=gcps,
    basemap_path=basemap_path,
    output_path=visualization_path,
    title="Qualicum Beach Ground Control Points",
    point_size=100,
    point_color='red',
    show_labels=True
)

print(f"\nVisualization saved to: {visualization_path}")


## 5. Alternative: Use Esri World Imagery Basemap


In [ ]:
# Download Esri World Imagery basemap (satellite imagery)
basemap_esri_path = str(output_dir / "qualicum_beach_basemap_esri.tif")

print("Downloading basemap from Esri World Imagery...")
basemap_esri_path = download_basemap(
    bbox=bbox,
    output_path=basemap_esri_path,
    source="esri_world_imagery",
    zoom=None
)

# Visualize with Esri basemap
visualization_esri_path = str(output_dir / "qualicum_beach_gcps_visualization_esri.png")

visualize_gcps_on_basemap(
    gcps=gcps,
    basemap_path=basemap_esri_path,
    output_path=visualization_esri_path,
    title="Qualicum Beach Ground Control Points (Esri World Imagery)",
    point_size=100,
    point_color='yellow',
    show_labels=True
)

print(f"\nVisualization saved to: {visualization_esri_path}")


## 6. Summary Statistics


In [ ]:
print("\n=== GCP Summary ===")
print(f"Total GCPs: {len(gcps)}")

if gcps:
    lats = [gcp['lat'] for gcp in gcps]
    lons = [gcp['lon'] for gcp in gcps]
    elevations = [gcp.get('z', 0) for gcp in gcps]
    
    print(f"\nLatitude range: {min(lats):.6f} to {max(lats):.6f}")
    print(f"Longitude range: {min(lons):.6f} to {max(lons):.6f}")
    
    if any(elevations):
        print(f"Elevation range: {min(elevations):.2f} to {max(elevations):.2f} m")
    
    print(f"\nBounding box:")
    print(f"  Min: ({min_lat:.6f}, {min_lon:.6f})")
    print(f"  Max: ({max_lat:.6f}, {max_lon:.6f})")
